In [ ]:
import os
import random
from pydub import AudioSegment
from shutil import copyfile

In [ ]:
def get_random_background_segment(background_sounds, segment_length):
    bg_sound = random.choice(background_sounds)
    start_ms = random.randint(0, len(bg_sound) - segment_length)
    return bg_sound[start_ms:start_ms + segment_length]

def load_background_sounds(parent_folder):
    background_folders = ["rain", "wind", "water"]
    background_sounds = []
    for folder in background_folders:
        for sound_file in os.listdir(os.path.join(parent_folder, folder)):
            file_extension = os.path.splitext(sound_file)[-1].lower()
            if file_extension in ['.wav', '.mp3']:
                sound_path = os.path.join(parent_folder, folder, sound_file)
                background_sounds.append(AudioSegment.from_file(sound_path, format=file_extension[1:]))
    return background_sounds

In [ ]:
# Paths
parent_folder_name = "/Users/ankush/Downloads/deakin-units/data/Overlay_Data"
audio_chunks_folder = "/Users/ankush/Downloads/deakin-units/data/final_training"
output_folder = "/Users/ankush/Downloads/deakin-units/data/combined_audio"
os.makedirs(output_folder, exist_ok=True)

background_sounds = load_background_sounds(parent_folder_name)


In [ ]:
# Determine the class with the maximum number of samples
file_counts = {d: len(os.listdir(os.path.join(audio_chunks_folder, d))) for d in os.listdir(audio_chunks_folder) if os.path.isdir(os.path.join(audio_chunks_folder, d))}
max_count = max(file_counts.values())

augmentations_done = {}  # Keep track of how many augmentations are done for each class


In [12]:

# For each class directory
for class_dir, count in file_counts.items():
    class_dir_path = os.path.join(audio_chunks_folder, class_dir)
    all_audio_files = [f for f in os.listdir(class_dir_path) if f.endswith('.wav')]
    
    # First, copy all original files to the output directory
    for audio_file in all_audio_files:
        origin_path = os.path.join(class_dir_path, audio_file)
        dest_path = os.path.join(output_folder, class_dir, audio_file)
        os.makedirs(os.path.join(output_folder, class_dir), exist_ok=True)
        copyfile(origin_path, dest_path)

    augmentations_done[class_dir] = 0  # Initialize counter for this class
    for i in range(max_count - count):  # Number of augmentations needed
        audio_file = random.choice(all_audio_files)
        audio_path = os.path.join(class_dir_path, audio_file)
        audio_chunk = AudioSegment.from_file(audio_path)

        bg_segment = get_random_background_segment(background_sounds, len(audio_chunk)).apply_gain(0.5)  # Reduce volume for subtlety
        combined = bg_segment.overlay(audio_chunk)

        # To ensure uniqueness in output file name
        output_file_name = f'combined_{i}_{audio_file}'
        output_path = os.path.join(output_folder, class_dir, output_file_name)
        combined.export(output_path, format="wav")

        augmentations_done[class_dir] += 1  # Increment the counter

# Print out the augmentations done for verification
for class_dir, count in augmentations_done.items():
    print(f"Augmented {count} files for class: {class_dir}")

# At this point, verify manually or programmatically that each class in the output directory has the same number of files


Augmented 725 files for class: Malleefowl
Augmented 797 files for class: Vanellus miles
Augmented 653 files for class: Spotted Nightjar
Augmented 740 files for class: Grey Teal
Augmented 641 files for class: Pheasant Coucal
Augmented 557 files for class: Tiger quoll
Augmented 0 files for class: coyote
Augmented 377 files for class: Peaceful Dove
Augmented 636 files for class: capra hirus
Augmented 722 files for class: Pacific Swift
Augmented 661 files for class: Orange footed Scrubfowl
Augmented 56 files for class: Large-tailed Nightjar
Augmented 570 files for class: Little Bronze Cuckoo
Augmented 698 files for class: Felis Catus
Augmented 627 files for class: common pheasant
Augmented 547 files for class: Australian Lyrebird
Augmented 312 files for class: Uperoleia laevigata
Augmented 298 files for class: Sus scrofa Wild Pig
Augmented 781 files for class: Australian Brushturkey
Augmented 780 files for class: Rattus
Augmented 673 files for class: Pacific Koel
